## Login to Low-side domain as a data scientist

In [1]:
import syft as sy
import pandas as pd

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/carmenpopa/.hagrid/quickstart


In [3]:
guest_domain_client = sy.login(url='http://localhost:8080', email='ana.banana@uni.org', password='student')
guest_domain_client

ConnectionError: Failed to fetch http://localhost:8080/api/v1/new/metadata. Response returned with code 404

## Check the datasets in the domain

In [3]:
guest_domain_client.datasets

,type,id,name,url
0,syft.core.node.new.dataset.Dataset,ed0bc2ddc01a48759b66a643c1ded1f3,Bubble tea mock data,None


In [4]:
dataset = guest_domain_client.datasets[0]
dataset

```python
Syft Dataset: Bubble tea mock data
Assets:
	mock_bubble_tea_data: Mock data for bubble tea consumption
Citation: Carmen Popa
Description: The fake data to show bubble tea trends for consumers

```

We can access the dataset assets (a.k.a the actual data) by running the following command.

In [5]:
dataset.assets

,key,type,id
0,mock_bubble_tea_data,syft.core.node.new.dataset.Asset,a650739e5294475f9b3aa72b2e5b4dc7


In [6]:
mock = dataset.assets[0].mock
print(mock.shape)

mock_df = mock.syft_action_data
mock_df.head()

(4, 4)


,drink_name,sugar_level,has_pearls,customer_ratings
0,brown_sugar,0.7,True,4.9
1,thai_milk,0.3,True,4.6
2,mango_coconut,0.9,False,3.5
3,strawberry_cheese,0.5,True,4.3


In [7]:
mock_df[0:2]

,drink_name,sugar_level,has_pearls,customer_ratings
0,brown_sugar,0.7,True,4.9
1,thai_milk,0.3,True,4.6


In [8]:
mock_df["sugar_level"].max()

0.9

## Writing queries

 **Which bubble tea flavour has the best customer ratings?**

At first, let's check the true result in the mock data so that we can validate our custom function later on. 

In [9]:
mock_df.iloc[mock_df['customer_ratings'].argmax()]["drink_name"]

'brown_sugar'

Now that we know the true result, we will create our custom function which, if approved, will be used on real data on the high-side domain.

In [10]:
@sy.syft_function(input_policy=sy.ExactMatch(drink_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def most_liked_bubble_tea(drink_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    # your code:
    df = drink_data
    most_liked_drink_row = df.iloc[mock_df['customer_ratings'].argmax()]
    return (# str(most_liked_drink_row["drink_name"]),
            float(most_liked_drink_row["customer_ratings"]),
            float(noise)
           )

Let's verify that the function and policy works on the mock data and returns the expected result.

In [11]:
result = most_liked_bubble_tea(drink_data=mock)
result

(4.9, -0.9322732645142884)

The function works as expected.

## Inspecting the attributes of the code & policy

In [12]:
most_liked_bubble_tea.kwargs

{NodeView(node_name='Adoring Sophia', verify_key=04dc3a4bf0da709524f2577ffe633c27b0fddbbe4071572ca604c6d25aeb07ce): {'drink_data': <UID: 4fbdf847d3bd41aeb5554b4967851929>}}

In [13]:
most_liked_bubble_tea.input_policy_type

syft.core.node.new.policy.ExactMatch

In [14]:
most_liked_bubble_tea.output_policy_type

syft.core.node.new.policy.OutputPolicyExecuteOnce

In [15]:
most_liked_bubble_tea.code

'@sy.syft_function(input_policy=sy.ExactMatch(drink_data=mock),\n                  output_policy=sy.SingleExecutionExactOutput())\ndef most_liked_bubble_tea(drink_data):\n    import pandas as pd\n    from opendp.mod import enable_features\n    enable_features(\'contrib\')\n    from opendp.measurements import make_base_laplace\n    aggregate = 0.\n    base_lap = make_base_laplace(scale=5.)\n    noise = base_lap(aggregate)\n\n    # your code:\n    df = drink_data\n    most_liked_drink_row = df.iloc[mock_df[\'customer_ratings\'].argmax()]\n    return (# str(most_liked_drink_row["drink_name"]),\n            float(most_liked_drink_row["customer_ratings"]),\n            float(noise)\n           )\n'

## Submit code for review on the Data Owner side

In [16]:
new_project = sy.Project(name="My first query on bubble tea data")
proj_desc = """Hi, I want to know which bubble tea flavour is the most liked"""

In [17]:
guest_domain_client.api.services.code.request_code_execution(most_liked_bubble_tea)

<class 'syft.core.node.new.response.SyftError'>: documents must have only string keys, key was NodeView(node_name='Adoring Sophia', verify_key=04dc3a4bf0da709524f2577ffe633c27b0fddbbe4071572ca604c6d25aeb07ce)

In [21]:
guest_domain_client.api.services.code.request_code_execution(most_liked_bubble_tea)
guest_domain_client.code

In [27]:
new_project.set_description(proj_desc)
submitted_code = guest_domain_client.code[0]
submitted_code

IndexError: list index out of range

In [30]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)
new_project.changes

,type,id
0,syft.core.node.new.project.ObjectPermissionChange,ca4e9d41f0d84ea7a54dd33c999a65db


In [31]:
new_project

```python
class ProjectSubmit:
  id: str = None
  name: str = "My first query on Dailymotion mock data"
  description: str = "Hi, I want to know which algorithm is most freqently being used for video recommendation."
  changes: str = [syft.core.node.new.project.ObjectPermissionChange]

```

Now we are ready to submit the code.

In [32]:
guest_domain_client.submit_project(new_project)

<class 'syft.core.node.new.response.SyftSuccess'>: Project Submitted

**At this point, we will go back to the data-owner-workflow notebook to review the submitted code and either approve or reject it.**